<a href="https://colab.research.google.com/github/mansine/Electricity-Consumption/blob/main/Data%20preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,SimpleRNN
from math import sqrt
import math
from numpy import split
import numpy
from numpy import array
from numpy import mean
from numpy.linalg import eig
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """


In [2]:
#reading the dataset
df=pd.read_csv("/content/File1.txt",header = None,delimiter = ' ')
#adding columns in the data
df.columns = ['MeterID','DateTime','ElectricityConsumption']
#checking the range of DateTime column
df['DateTime'].min() , df['DateTime'].max()
#Keep the DataFrame with valid entries in the same variable.
df.dropna(inplace=True)
#seperating days and time from 5 digits DateTime columns
#Converting DateTime datatype from integer to string
df['DateTime'] = df['DateTime'].astype('str')
new = df['DateTime'].str.split("", n =0, expand = True)
#First 2digits are days in 2year readings
df['date'] = new[1] + new[2] + new[3] 
#Last 2digits showing time code
df['time_in_minutes'] = new[4] + new[5]
#No more need of duplicated data
df.drop('DateTime', axis=1, inplace=True)
#convert string to integer
df['date'] = df['date'].astype(int)
df['time_in_minutes'] = df['time_in_minutes'].astype(int)
# convert the mins to real time by multiplying it by 30 mins
df['time_in_minutes']=df['time_in_minutes']*30
df.isnull().sum()
#sorting the value in ascending order by MererID,date and time_in_minutes
df.sort_values(['MeterID','date','time_in_minutes'],ascending=True,inplace=True)
df.reset_index(drop=True,inplace=True)

In [3]:
#covert minutes data into hours
df['hours']=pd.to_datetime(df.time_in_minutes, unit='m').dt.strftime('%H:%M')
#coverting days of year into actual date
year=2009
df['Date']=pd.to_datetime(df.date,unit='D', origin=str(year))
# Drop unwanted attributes
df.drop('time_in_minutes', axis=1, inplace=True)
df.drop('MeterID', axis=1, inplace=True)
df.drop('date',axis=1,inplace=True)
#kilowatt consumption per day is aggregated
df=df.groupby(['Date','hours']).agg('sum')

In [4]:
df.head(10)

ElectricityConsumption
Date       hours                        
2009-07-15 00:00                 531.176
           00:30                 464.058
           01:00                 400.316
           01:30                 353.643
           02:00                 320.181
           02:30                 293.990
           03:00                 283.929
           03:30                 270.338
           04:00                 261.528
           04:30                 268.655

In [5]:
#NEW DATA
df.to_csv('ElectricityConsumptionSum.csv')



```
# This is formatted as code
```

# New Data

In [6]:
df=pd.read_csv("/content/ElectricityConsumptionSum.csv")


In [7]:
import plotly.graph_objects as go
trace1 = go.Scatter(
            y=df.ElectricityConsumption,
            x = df.Date,
    mode = 'lines',
    name = 'Actual'
)
layout = go.Layout(
    title = "Actual Consumption",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Electricity Consumption"}
)
fig = go.Figure(data=[trace1], layout=layout)
fig.show()